In [ ]:
from pyqubo import Integer
W = 20
c = {0:5, 1:7,  2:2, 3:1, 4:4, 5:3}
w = {0:8, 1:10, 2:6, 3:4, 4:5, 5:3}
N = len(w)

In [ ]:
from pyqubo import Array, LogEncInteger

x = Array.create('x', shape=(N), vartype='BINARY')
y = LogEncInteger("y", (0, W))

In [6]:
class LogEncInteger(Integer):
    def __init__(self, label, lower, upper):
        assert upper > lower, "upper value should be larger than lower value"
        assert isinstance(lower, int)
        assert isinstance(upper, int)

        self.lower = lower
        self.upper = upper
        self._num_variables = int(np.log2(upper - lower))+1
        self.array = Array.create(label, shape=self._num_variables, vartype='BINARY')
        self.label = label
        self._express = lower + sum(x*2**i for i, x in enumerate(self.array))

In [7]:
from pyqubo import Constraint

key1 = max(c, key = lambda k: c[k])
B = 40
A = 10 * B * c[key1]

HA = Constraint(
    A * ( W - sum( w[a] * x[a] for a in range(N) ) - y )**2
    , label='HA'
)

HB = - B * sum( c[a] * x[a] for a in range(N) )

In [8]:
from pyqubo import solve_qubo
import dimod
import math

print("[Inputs]")
print()
print("W (ナップサックの容量) : "+str(W/10)+"kg")
print("N (宝物の数): "+str(N))
print()
print("weight list")
print(w)
print()
print("cost list")
print(c)
print()
print("A : "+str(A))
print("B : "+str(B))

H = HA + HB
Q = H
model = Q.compile()
q, offset = model.to_qubo()

sampleset = dimod.ExactSolver().sample_qubo(q)
decoded_sample  = model.decode_sample(sampleset.first.sample, vartype="BINARY")
print()
print("[Results]")
print()
print("decoded_sample.sample:")
print(decoded_sample.sample)
print()
print("x (選ばれた宝物) :")

treasures = ['A','B','C','D','E','F','G']
weight = 0
cost = 0

for k in range(N):
    if decoded_sample.array('x', k) != 0:
        print("宝物"+treasures[k])
        weight += w[k]
        cost += c[k]


sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem)) for elem in range(math.ceil(math.log2(W)))])

print()
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("合計の重さ : "+str(weight/10)+"kg")
print("合計の価格 : $"+str(cost)+",000")

[Inputs]

W (ナップサックの容量) : 2.0kg
N (宝物の数): 6

weight list
{0: 8, 1: 10, 2: 6, 3: 4, 4: 5, 5: 3}

cost list
{0: 5, 1: 7, 2: 2, 3: 1, 4: 4, 5: 3}

A : 2800
B : 40

[Results]

decoded_sample.sample:
{'y[3]': 0, 'y[2]': 0, 'y[1]': 1, 'y[0]': 0, 'y[4]': 0, 'x[5]': 1, 'x[4]': 1, 'x[3]': 0, 'x[2]': 0, 'x[1]': 1, 'x[0]': 0}

x (選ばれた宝物) :
宝物B
宝物E
宝物F

スラック変数Y = 2

broken
{}
合計の重さ : 1.8kg
合計の価格 : $14,000
